In [1]:
using ProgressMeter: @showprogress

using BayesianExperiments

In [247]:
function simulate(δ, n, ;r=0.707, thresh=9, minsample=20, stepsize=1)
    # we will use two-sided decision rule for bayes factor
    #rule = TwoSidedBFThresh(thresh)
    rule = OneSidedBFThresh(thresh)
    
    # the prior distribution of effect size,
    # r is the standard deviation
#     model = StudentTEffectSize(r=r)
    model = NormalEffectSize(0.0, r)
    
    # setup the experiment
    experiment = ExperimentBF(model=model, rule=rule)
    
    # create a sample with size n, the effect size is 
    # specified as δ
    xs = rand(Normal(δ, 1), n)
    
    i = 0
    # specify the stopping condition
    while (i < n) & (experiment.winner === nothing)
        i += stepsize
        
        # if minimum number of sample is not reached, 
        # keep collecting data
        if i < minsample
            continue
        end
        
        stats = NormalStatistics(xs[1:i])
        experiment.stats = stats
        decide!(experiment)
    end
    experiment
end

simulate (generic function with 1 method)

In [266]:
ns = 1000
n = 50_000*7
minsample = 50000
r = 0.1
thresh = 3
stepsize=50000

deltas = zeros(ns)

winners1 = []
samplesizes1 = []
@showprogress for i in 1:ns
    experiment = simulate(deltas[i], n; r=r, thresh=thresh, minsample=minsample, stepsize=stepsize)
    push!(winners1, experiment.winner)
    push!(samplesizes1, experiment.stats.n)
end

rej1 = sum(winners1 .== "alternative")

Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


10

In [264]:
deltas = ones(ns) * 0.006
#deltas = rand(Normal(0, 0.01), ns);

winners2 = []
samplesizes2 = []
@showprogress for i in 1:ns
    experiment = simulate(deltas[i], n; r=r, thresh=thresh, minsample=minsample, stepsize=stepsize)
    push!(winners2, experiment.winner)
    push!(samplesizes2, experiment.stats.n)
end

rej2 = sum(winners2 .== "alternative")

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


677

In [265]:
@show type1_error = rej1 / ns;
@show power = rej2 / ns;
@show fdr = rej1 / (rej1 + rej2);
@show mean((samplesizes1 .+ samplesizes2)./2)

type1_error = rej1 / ns = 0.012
power = rej2 / ns = 0.677
fdr = rej1 / (rej1 + rej2) = 0.01741654571843251
mean((samplesizes1 .+ samplesizes2) ./ 2) = 301700.0


301700.0

The sample size calulated using Frequentist way:

https://www.evanmiller.org/ab-testing/sample-size.html

- base conversion rate: 9%
- minimum detectable effect: 2%
- power 1 - $\beta$=65%
- significance level $\alpha$ = 2%
- sample size: 372216

In [232]:
@show type1_error = rej1 / ns;
@show power = rej2 / ns;
@show fdr = rej1 / (rej1 + rej2);
@show mean((samplesizes1 .+ samplesizes2)./2)

type1_error = rej1 / ns = 0.023
power = rej2 / ns = 0.551
fdr = rej1 / (rej1 + rej2) = 0.04006968641114982
mean((samplesizes1 .+ samplesizes2) ./ 2) = 343925.0


343925.0

In [308]:
psd = pooledsd(293, 289, 321626, 35569)

292.6041457250259

In [134]:
twostats = TwoNormalStatistics(
    NormalStatistics(meanx=203, n=976400, sdx=1072),
    NormalStatistics(meanx=215, n=107420, sdx=1120)
)

2-element StaticArrays.SArray{Tuple{2},NormalStatistics,1,2} with indices SOneTo(2):
 NormalStatistics(976400, 203, 1072)
 NormalStatistics(107420, 215, 1120)

In [135]:
es = effectsize(twostats)

-0.011143583479134958

In [139]:
model = StudentTEffectSize(r=0.1)
bayesfactor(model, StudentTStatistics(twostats))

10.29875719657946

In [138]:
model = NormalEffectSize(0.0, 0.1)
bayesfactor(model, twostats)

12.993292520612217

In [225]:
p = 0.09

0.09

In [227]:
p*0.02/ sqrt(p*(1-p))

0.006289709020331509